In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import joblib

tqdm.pandas()

In [181]:
df = pd.read_csv (r'biblioteca/bbc-text.csv')

In [3]:
df

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...
...,...,...
2220,business,cars pull down us retail figures us retail sal...
2221,politics,kilroy unveils immigration policy ex-chatshow ...
2222,entertainment,rem announce new glasgow concert us band rem h...
2223,politics,how political squabbles snowball it s become c...


In [109]:
train = pd.read_csv("biblioteca/bbc-text-train.csv", usecols=["category", "text"])
valid = pd.read_csv("biblioteca/bbc-text-validation.csv", usecols=["category", "text"])
test = pd.read_csv("biblioteca/bbc-text-test.csv", usecols=["category", "text"])

In [110]:
def strip_trash(df, column="text"):
    df[column] = df[column].str.strip('{}"')
    return df

In [111]:
def map_category(df):
    df = df.dropna()
    df["category"] = df['category'].map({"tech": 0, "business": 1, "sport":2, "entertainment": 3, "politics": 4 })
    return df

In [112]:
train_clean = strip_trash(train)
valid_clean = strip_trash(valid)
test_clean = strip_trash(test); len(train)

1113

In [113]:
train_clean = map_category(train_clean)
valid_clean = map_category(valid_clean)
test_clean = map_category(test_clean)

train_clean = train_clean.dropna()
train_clean.category.unique()

array([2., 1., 3., 0., 4.])

In [114]:
x_train, y_train = train_clean["text"].values, train_clean["category"].values

In [115]:
x_valid, y_valid = valid_clean["text"].values, valid_clean["category"].values

In [116]:
x_test, y_test = test_clean["text"].values, test_clean["category"].values

In [117]:
len(x_train), len(y_train), len(x_valid), len(y_valid), len(x_test), len(y_test)

(1112, 1112, 551, 551, 556, 556)

In [118]:
del(train_clean)
del(valid_clean)
del(test_clean)

In [131]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from xgboost.sklearn import XGBClassifier
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer

In [132]:
model = xgb.XGBClassifier()

In [143]:
pipeline = Pipeline((
    ("vectorizer", TfidfVectorizer(ngram_range=(1, 3), sublinear_tf=True,
                                   min_df=2, max_df=0.5)),
    ("model", model)
))

In [144]:
param_grid = {
    "model__max_depth": [3, 4, 5],
    "model__n_estimators": [100, 300, 1000, 3000]
}

In [145]:
grid = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1, scoring='roc_auc')

In [146]:
grid.fit(x_train, y_train)

/home/maria/.asdf/installs/python/3.10.4/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/maria/.asdf/installs/python/3.10.4/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/maria/.asdf/installs/python/3.10.4/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 219, in __call__
    return self._score(
  File "/home/maria/.asdf/installs/python/3.10.4/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 352, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
/home/maria/.asdf/installs/python/3.10.4/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring

GridSearchCV(cv=5,
             estimator=Pipeline(steps=(('vectorizer',
                                        TfidfVectorizer(max_df=0.5, min_df=2,
                                                        ngram_range=(1, 3),
                                                        sublinear_tf=True)),
                                       ('model',
                                        XGBClassifier(base_score=None,
                                                      booster=None,
                                                      callbacks=None,
                                                      colsample_bylevel=None,
                                                      colsample_bynode=None,
                                                      colsample_bytree=None,
                                                      early_stopping_rounds=None,
                                                      enable_categorical=False,
                                                      eval_metric=None,
                                                      gamma=None, gpu_...
                                                      max_cat_to_onehot=None,
                                                      max_delta_step=None,
                                                      max_depth=None,
                                                      max_leaves=None,
                                                      min_child_weight=None,
                                                      missing=nan,
                                                      monotone_constraints=None,
                                                      n_estimators=100,
                                                      n_jobs=None,
                                                      num_parallel_tree=None,
                                                      predictor=None,
                                                      random_state=None,
                                                      reg_alpha=None,
                                                      reg_lambda=None, ...)))),
             n_jobs=-1,
             param_grid={'model__max_depth': [3, 4, 5],
                         'model__n_estimators': [100, 300, 1000, 3000]},
             scoring='roc_auc')

In [151]:
from sklearn.metrics import classification_report, accuracy_score

In [190]:
target_names = grid.classes_

In [191]:
new_target = [[int(i)] for i in target_names]

In [192]:
label_names = ['tech', 'business', 'sport', 'entertainment', 'politics']

In [193]:
preds_valid = grid.predict(x_valid)

In [194]:
preds_valid = grid.predict(x_valid)
print(classification_report(y_valid, preds_valid, target_names=label_names, digits=4))


               precision    recall  f1-score   support

         tech     0.9417    0.9417    0.9417       103
     business     0.9375    0.9375    0.9375       128
        sport     0.9846    0.9846    0.9846       130
entertainment     0.9326    0.9881    0.9595        84
     politics     0.9505    0.9057    0.9275       106

     accuracy                         0.9510       551
    macro avg     0.9494    0.9515    0.9502       551
 weighted avg     0.9512    0.9510    0.9509       551



In [195]:
print(accuracy_score(y_valid, preds_valid))

0.9509981851179673
